在第9.3节中，我们介绍了马尔可夫模型和n-gram语言模型，其中时间步骤t处的标记的条件概率仅取决于前n个标记。如果我们想考虑早于时间步骤t的标记对t的可能影响，我们需要增加n。然而，模型参数的数量也会随着n的增加而呈指数级增长，因为对于一个词汇集V，我们需要存储V^n个数字。因此，与建模P(x_t | x_{t-1}, ..., x_{t-n})相比，使用潜在变量模型更可取。

(9.4.1)
其中h_t是一个隐藏状态，存储了直到时间步骤t的序列信息。一般来说，任何时间步骤t的隐藏状态可以基于当前输入x_t和前一隐藏状态h_{t-1}计算得出：

(9.4.2)
对于(9.4.2)中足够强大的函数f，潜在变量模型并不是一种近似。毕竟，h_t可以简单地存储到目前为止观察到的所有数据。然而，它可能会使计算和存储变得昂贵。

回顾一下我们在第5节中讨论的具有隐藏单元的隐藏层。值得注意的是，隐藏层和隐藏状态指的是两个非常不同的概念。隐藏层是在输入到输出的路径上被隐藏的层，如前所述。从技术上讲，隐藏状态是我们在给定步骤执行的操作的输入，只能通过查看先前时间步长的数据来计算。

循环神经网络（RNN）是具有隐藏状态的神经网络。在介绍RNN模型之前，我们首先回顾一下第5.1节中介绍的MLP模型。

MLP（多层感知器）是一种前馈神经网络，它包含一个输入层、一个或多个隐藏层和一个输出层。每一层都由一个或多个神经元组成，这些神经元与前一层和后一层的神经元相连接。在MLP中，信息沿着从输入层到输出层的单向路径流动。激活函数通常是非线性的，例如ReLU或Sigmoid函数。

与MLP不同，循环神经网络（RNN）在序列数据上表现更好，因为它们能够捕获序列中的时间依赖关系。RNN的关键特性是隐藏状态，它允许网络在处理序列时保留先前时间步长的信息。这使得RNN在处理涉及时间依赖关系的任务（如语言建模、时间序列预测等）时非常有效。然而，RNN也存在一些挑战，例如梯度消失和梯度爆炸问题，这可能会影响训练的稳定性和性能。为了解决这些问题，研究人员提出了一些改进的RNN结构，如长短时记忆网络（LSTM）和门控循环单元（GRU）。

In [1]:
import torch
from d2l import torch as d2l

让我们看一个具有单个隐藏层的MLP。设隐藏层的激活函数为ϕ。给定一个批量大小为n且有d个输入的样本小批量X，隐藏层输出H的计算方式为：

(9.4.3)
在(9.4.3)中，我们有权重参数W₁，偏置参数b₁，以及隐藏层的隐藏单元数h。有了这些参数，我们在求和过程中应用广播（参见第2.1.4节）。接下来，隐藏层输出H被用作输出层的输入，计算方式为：

(9.4.4)
其中O是输出变量，W₂是权重参数，b₂是输出层的偏置参数。如果是分类问题，我们可以使用O来计算输出类别的概率分布。

这与我们在第9.1节中解决的回归问题完全类似，因此我们省略了细节。简言之，我们可以随机选择特征-标签对，并通过自动微分和随机梯度下降来学习网络的参数。
''' 

''' 
当我们有隐藏状态时，情况就完全不同了。让我们更详细地看一下这个结构。

假设我们在时间步骤t处有一个输入小批量X_t。换句话说，对于一个有n个序列样本的小批量，X_t的每一行对应于序列中时间步骤t处的一个样本。接下来，用H_t表示时间步骤t的隐藏层输出。与MLP不同，这里我们保存前一时间步骤的隐藏层输出H_{t-1}，并引入一个新的权重参数W_{hh}来描述如何在当前时间步骤中使用前一时间步骤的隐藏层输出。具体来说，当前时间步骤的隐藏层输出的计算由当前时间步骤的输入和前一时间步骤的隐藏层输出共同决定：

(9.4.5)
与(9.4.3)相比，(9.4.5)多了一个项H_{t-1}W_{hh}，因此实例化了(9.4.2)。从相邻时间步骤的隐藏层输出H_t和H_{t-1}之间的关系，我们知道这些变量捕获并保留了序列在当前时间步骤的历史信息，就像神经网络当前时间步骤的状态或记忆一样。因此，这样的隐藏层输出称为隐藏状态。由于隐藏状态在当前时间步骤中使用前一时间步骤的相同定义，因此(9.4.5)的计算是循环的。因此，我们说，基于循环计算的隐藏状态的神经网络被称为循环神经网络。在RNN中执行(9.4.5)计算的层称为循环层。

有许多不同的方法可以构建RNN。具有由(9.4.5)定义的隐藏状态的RNN非常常见。对于时间步骤t，输出层的输出与MLP中的计算类似：

(9.4.6)
RNN的参数包括隐藏层的权重W_{xh}和偏置b_h，以及输出层的权重W_hy和偏置b_y。值得一提的是，即使在不同的时间步骤，RNN始终使用这些模型参数。因此，RNN的参数化成本不会随着时间步数的增加而增加。

图9.4.1展示了三个相邻时间步骤中RNN的计算逻辑。在任何时间步骤t，隐藏状态的计算可以被视为：(i)连接当前时间步骤t的输入X_t和前一时间步骤t-1的隐藏状态H_{t-1}；(ii)将连接结果输入到具有激活函数ϕ的全连接层中。这个全连接层的输出就是当前时间步骤t的隐藏状态H_t。在这种情况下，模型参数是(9.4.5)中的W_{xh}和W_{hh}的连接，以及偏置b_h。当前时间步骤t的隐藏状态H_t将参与计算下一个时间步骤t+1的隐藏状态H_{t+1}。此外，H_t还将被输入到全连接输出层中，以计算当前时间步骤t的输出O_t。

../_images/rnn.svg
![](https://d2l.ai/_images/rnn.svg)
图9.4.1 具有隐藏状态的RNN。

我们刚刚提到，计算隐藏状态H_t的方法等价于将X_t和H_{t-1}连接起来，然后将W_{xh}和W_{hh}连接起来进行矩阵乘法。尽管这可以从数学上证明，但在下面我们只是

In [2]:


X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[-0.1937,  0.8541,  0.8771, -1.8238],
        [ 1.2450, -2.0936, -3.9491, -5.1588],
        [ 1.3786, -1.5542, -1.8481,  1.1853]])

In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[-0.1937,  0.8541,  0.8771, -1.8238],
        [ 1.2450, -2.0936, -3.9491, -5.1588],
        [ 1.3786, -1.5542, -1.8481,  1.1853]])

这两个`torch.matmul`操作的结果相同，因为它们都计算了相同的矩阵乘法。为了更好地理解这一点，让我们详细解释每个操作：

1. `torch.matmul(X, W_xh) + torch.matmul(H, W_hh)`：
   - 首先，我们计算`X`与`W_xh`的矩阵乘法，结果是一个形状为(3, 4)的矩阵。
   - 然后，我们计算`H`与`W_hh`的矩阵乘法，结果也是一个形状为(3, 4)的矩阵。
   - 最后，我们将这两个矩阵相加，得到一个形状为(3, 4)的矩阵。

2. `torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))`：
   - 首先，我们将`X`和`H`沿第1轴（列）连接起来，得到一个形状为(3, 5)的矩阵。
   - 然后，我们将`W_xh`和`W_hh`沿第0轴（行）连接起来，得到一个形状为(5, 4)的矩阵。
   - 最后，我们计算这两个连接后的矩阵的矩阵乘法，得到一个形状为(3, 4)的矩阵。

现在，让我们观察第二个操作的矩阵乘法是如何等价于第一个操作的两个矩阵乘法之和的。在第二个操作中，我们将`X`和`H`连接在一起，然后将`W_xh`和`W_hh`连接在一起。当我们执行矩阵乘法时，实际上是将`X`与`W_xh`相乘，将`H`与`W_hh`相乘，然后将结果相加。这正是第一个操作所做的。

因此，这两个`torch.matmul`操作得到相同的结果，因为它们都计算了相同的矩阵乘法。

1. 如果我们使用RNN来预测文本序列中的下一个字符，所需的输出维度将等于词汇表的大小。这是因为我们通常使用one-hot编码来表示每个字符，这样的编码方式会为词汇表中的每个字符分配一个唯一的向量。在预测阶段，RNN的输出将通过softmax函数转换为概率分布，表示生成每个可能字符的概率。

2. RNN可以表达基于文本序列中所有先前标记的某个时间步的标记的条件概率，因为它们具有“记忆”功能。在RNN中，每个时间步的隐藏状态都取决于当前输入和前一时间步的隐藏状态。这意味着，在每个时间步，RNN都能访问到此前所有时间步的信息。因此，RNN能够捕获序列中的长期依赖关系，从而表达基于所有先前标记的标记的条件概率。

3. 如果通过一个长序列进行反向传播，可能会遇到梯度消失或梯度爆炸的问题。具体来说，当序列很长时，梯度可能会在反向传播过程中变得非常小（梯度消失）或非常大（梯度爆炸）。这可能会导致网络难以学习，并可能导致不稳定的训练过程。为了解决这个问题，研究人员提出了一些改进的RNN结构，如长短期记忆网络（LSTM）和门控循环单元（GRU）。

4. 这一节中描述的语言模型可能存在一些问题。首先，字符级语言模型可能无法充分捕获语言的语义结构，因为它们在字符级别而不是单词级别处理文本。其次，基本的RNN模型可能难以处理长序列，因为它们可能会遇到梯度消失或梯度爆炸的问题。此外，由于RNN在每个时间步都依赖于前一时间步的计算，因此它们难以并行化，这可能会限制训练速度。
